# Soustraction d'image moyenne sur une vidéo

## Code

In [6]:
import ipywidgets as widgets
import os
import cv2,numpy,json


def video2listImage(adresseVideo,staSec,stoSec): # Renvoie une liste d'image sous forme d'array
    video = cv2.VideoCapture(adresseVideo)
    videoFPS = video.get(cv2.CAP_PROP_FPS)
    print(videoFPS)
    
    # Première étape, aller au time code de départ
    continuer=True
    compteur=0
    while continuer:
        isRead = video.read()
        if not isRead:
            continuer = False
            print("Time code de départ trop grand")
            return None
        else:
            compteur+=1
            if (compteur/videoFPS>staSec):
                continuer = False
    
    compteur-=1
    if staSec!=0:
        print("Time code de début atteint",compteur/videoFPS)
    cptDebut=compteur
    
    #On commence à enregistrer les images
    imlist=[]
    continuer = True
        
    while continuer:
        isRead , frame = video.read()  # A-t-on lu une image et si oui cette image
        if not isRead:
            continuer = False
        else:
            imlist.append(frame)
            compteur+=1
            
            if (stoSec!=-1): #Si on ne va pas jusqu'à la fin de la vidéo
                if(compteur/videoFPS>stoSec):
                    print("Time code de fin atteint",compteur/videoFPS)
                    continuer=False
    
    if (compteur/videoFPS<stoSec):
        print("Time code de fin trop grand")
        return None
    duree=(compteur-cptDebut)/videoFPS
    
    
    return imlist,videoFPS,duree

def subImageMoyArr(imListArr,PosIm): # Renvoie un tableau OpenCV
    n=len(imListArr)
    imCentre=imListArr[PosIm]
    imMoy=calcImageMoyenneArr(imListArr)
    imSub = cv2.subtract(imCentre,imMoy)
    return imSub

def calcImageMoyenneArr(imListArr): # Renvoie array
    w,h=len(imListArr[0]), len(imListArr[0][0])
    arr=numpy.zeros((h,w,4),numpy.float)
    images = numpy.array([image for image in imListArr])
    # Round values in array and cast as 8-bit integer
    arr=numpy.array(numpy.mean(images,axis=(0)),dtype=numpy.uint8)
    return arr

def SubImageMoySurListe(imList,tailleMoy,PosIm): # Renvoie une liste des images soustraites
    imlistMoy = [0]*tailleMoy
    imlistSubMoy = [0]*(len(imList)-tailleMoy+1)
    for i in range(PosIm,len(imList)-(tailleMoy-PosIm)):
        # On selectionne les images autour de celle traitée
        for j in range(tailleMoy):
            imlistMoy[j]=imList[j+i-PosIm]
        imlistSubMoy[i-PosIm]=subImageMoyArr(imlistMoy,PosIm)
    return imlistSubMoy

#Enregistrement de la vidéo
def listImage2Video(imList,writeFPS,nomFichier):
    height,width,layers=imList[0].shape
    # choose codec according to format needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video=cv2.VideoWriter(nomFichier, fourcc, writeFPS,(width,height))
    for img in imList:
        video.write(img)
    return video,width,height


def soustractionDeFond(nomVideoOrigine):
    
    ## Recuperation du nom du fichier avec l'extension
    allfiles=os.listdir(os.getcwd()+"/Data/"+nomVideoOrigine)
    file=[name for name in allfiles if (name.startswith(nomVideo.value) and name[-4:] in [".mp4",".avi",".mov",".3gp"])][0]

    adr="Data/"+nomVideoOrigine+"/"+file

    ## -- Paramètres -- ##
    startSecond=0
    stopSecond=-1
    tailleIntervalle=13
    posIm=7
    posIm-=1 #position en pyhton (qui commence a zero)
    
    # Soustraction
    imList, FPS, duree = video2listImage(adr,startSecond,stopSecond) # Tableau d'array
    if (imList!=None):
        imListSubMoy=SubImageMoySurListe(imList, tailleIntervalle,posIm) # Les mêmes images, auxquelles on a enlevé les images moyennes
    
    # Enregistrement
    videoWriteFile="SUB_"+file
    os.chdir("Data/"+nomVideoOrigine)
    videoSub,width, height= listImage2Video(imListSubMoy,FPS, videoWriteFile)
    videoSub.release()

    # Génération du fichier JSON
    data ={}
    data["Nom"] = nomVideoOrigine
    data["height"] = height
    data["width"] = width
    data["Frame per second"] = FPS
    data["Duree"] = duree
    data["Taille intervalle image moyenne"] = tailleIntervalle
    data["Position de l'image dans l'intervale"] = posIm+1

    with open('data.json', 'w') as f:
     json.dump(data,f,indent=4, ensure_ascii=False, sort_keys=False)

    os.chdir("..")
    os.chdir("..")

    print("Done")

# On recupère tout les noms de dossier pour choisir la vidéo
alldir=os.listdir(os.getcwd()+"/Data")
listeVideo=[filename for filename in alldir]

nomVideo = widgets.Dropdown(options=listeVideo,
    description='Choisissez une vidéo :',
    disabled=False,
)
display(nomVideo)

out = widgets.Output()


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        out.clear_output()
        with out :
            if change['new']!="0":
                print("Vous avez choisi %s" % change['new'])
                soustractionDeFond(change['new'])
            
nomVideo.observe(on_change)
display(out)

Dropdown(description='Choisissez une vidéo :', options=('0', 'Arc_Compil_Tir', 'Billard_Coup_1', 'Billard_Coup…

Output()

## Résultats

On rapelle que le cadre de la vidéo se doit d'être fixe pour utiliser ce programme.

La soustraction d'image moyenne permet bien d'effacer tout ce qui ne bouge pas. En revanche les joueurs reste très visibles, la détection de la balle ne peut donc pas reposer que sur cette soustraction. La soustraction de l'image moyenne présente cependant des inconvénients puisqu'elle floute légèrement la balle. Le paramètre tailleIntervalle permet de règler cet aspect. Pour les vidéos de tennis de table, la balle est généralement rapide et calculer l'image moyenne sur trois images consécutives suffit à la distinguer de ce qui est fixe.

In [2]:
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="SauvData/SUB_TT_echanges_1.mp4" type="video/mp4">
</video></div>""")

Sur les vidéos de billard, la soustraction d'image est particulièrement intéressante car elle permet d'identifier directement les boules mobiles. Les boules sont généralement plus lentes et un plus grand intervalle de calcul permet de mieux les identifier, notament au momment du choc.

In [3]:
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="SauvData/SUB_Billard_Coup_2.mp4" type="video/mp4">
</video></div>""")

Sur la vidéo de tir à l'arc en revanche, la flèche n'est pas distinguable, d'une part car elle est très fine et d'autre part car son mouvement filmé est très court temporellement.

In [4]:
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="SauvData/SUB_Arc_Compil_Tir.mp4" type="video/mp4">
</video></div>""")